In [584]:
strr = '''
'''

In [585]:
with open("/Users/hkmac/Downloads/input.txt") as f:
    strr = f.read()

In [586]:
inp = strr.strip().split("\n\n")

In [587]:
seq = inp[0].split(",")
seq = [int(x) for x in seq]

In [588]:
# seq

In [589]:
boards = [[[int(z) for z in y.split()] for y in x.split("\n")] for x in inp[1:]]